In [1]:
# Define IAM role
from sagemaker import get_execution_role
role = get_execution_role()

import Input_Config # Input Config - To be initialised
Input_Config.help()
bucket, W, x, y, vin_list, path, path2, path3, list_col, odo, ic = Input_Config._init_()

import Dummy_Config
Dummy_Config.help()
list_col = Dummy_Config._init_()

global bucket, W, x, y, vin_list, path, path2, path3, list_col, odo, ic

from d1 import path1cc, Tsim1, Tsim1pow
from statedwell import statedwell
from alertstat import alertstat
from d2 import s3_write
import Annotations
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

vinlist.csv not available
vin_list intialized with test list !!
bucket, W, x, y, vin_list, path, path2, path3, list_col, odo,ic
list_col


In [2]:
bucket="relai.poc.temp"
folder_base='saurabh/'
file_extn='.csv'

from d2 import get_vin_list
f1=get_vin_list(bucket=bucket, folder_base=folder_base, file_extn=file_extn)  #reading source
#f2=get_vin_list(bucket=bucket, folder_base='res/1/', file_extn=file_extn)  #reading already done
#f3= list( set(f1)-set(f2) ) # picking the remaining to do
f4=get_vin_list(bucket=bucket, folder_base='jcd/', file_extn=file_extn)  #reading source
f5=list(pd.read_excel('s3://{}/{}{}'.format(bucket,'0tmp/','vin_239.xlsx'))['vin'])
f6=f4+f5[0:40]
#path = 's3://{}/{}/{}.csv'.format(bucket,folder_base,vin)
#'s3://{}/{}{}.csv'.format(bucket,'alert/',vin)

In [ ]:
xy_data_list = []
file_counter=0
for vin in f6[69:]:  # .. # 4527-done .. 46 skipped .. FILES data corrupt!
    file_counter+=1
    print(file_counter, vin)
    file_periodic = 's3://{}/{}{}.csv'.format(bucket,folder_base,vin)
    file_alert    = 's3://{}/{}{}.csv'.format(bucket,'alert/',vin)
    cf1_p=0
    try:
        df1_p = pd.read_csv(file_periodic)
        df1_p['ENGINE_SPEED_x'] = df1_p['ENGINE_SPEED_y']
        df1_p['ENGINE_SPEED'] = df1_p['ENGINE_SPEED_y']
        cf1_p=1
    except:
        print(vin+" ---- Not Available")
        cf1_p=0
        
    if (cf1_p==1): 
        df3r=statedwell(W, df1_p)
        df1 = df1_p[df1_p['ENGINE_SPEED_x']>0]
        
    cf1_c=0
    try:
        df1_c = pd.read_csv(file_alert)
        cf1_c=1
    except:
        print(file_alert+'.csv'+"  Not Available")
        cf1_c=0
    
    if (cf1_c==1): df_temp0=alertstat(W, df1_p, df1_c)
    else : df_temp0=pd.DataFrame()
    
    
    from d2 import power
    df1 = power(df1)
    list_col = list(set(df1.columns) - set(['Unnamed: 0', 'GPS_SPEED_x',
                                             'ACT_WHL_SPEED','EVENT_UTC1', 'LATITUDE_x', 'LONGITUDE_x', 'ALTITUDE_x', 'HEADING_x',    
                                             'NO_OF_SAT_x', 'ENGINE_SPEED_y', 'WHEEL_SPEED_y', 'GPS_SPEED_y','IGNITION_STATUS_x', 'VEHICLE_ODO_x','GPS_ODO_x', 'ws1', 'odo', 'gear',
                                             'CLUTCH_PEDAL', 'BRAKE_PEDAL', 'ENGINE_HRS', 'DIST_TO_EMPTY', 's_ang','c_ang', 'alpha', 'trp', 'mass']
                                           )
                    )
    
    
    ldf=[]
    for i in list_col:
        if(i in ['power[-inf,0]','power[0,25]','power[25,75]','power[75,inf]']):
            df10 = Tsim1pow(P1=df1[i],odo=df1['odo'], P_name=i)
            ldf.append(df10)#.to_csv(i+".csv")
        else:
            df10 = Tsim1(P1=df1[i],odo=df1['odo'], P_name=i)
            ldf.append(df10)#.to_csv(i+".csv")
    
            
    if (df3r.shape[0]>0):
        if (df_temp0.shape[0]>0):
            df3r=pd.concat([df3r,df_temp0], axis=1)#, sort=False
        
    df_temp1=pd.concat(ldf, axis=1)#, sort=False
    df_temp2=pd.concat([df3r,df_temp1], axis=1)#, sort=False
    
    #----- Annotations ----- # skip-POC- Nirumal & Aseena taking care of populating display
    '''  
    col_L2_sys = Input_Config.col_L2_sys # name of column in jcd having L2-sys components
    if (df_temp2.shape[0]>0): 
        df_temp2=Annotations.Annotations_egr(df_temp2, col_L2_sys, path+'/1', vin) 
        xy_data_list.append(df_temp2)
        _=s3_write(bucket, 'res/1/{}.csv'.format(vin), df_temp2)
    '''
    _=s3_write(bucket, 'res/1/{}.csv'.format(vin), df_temp2)
    
    df_temp2['vin']=vin
    xy_data_list.append(df_temp2)
    # ---- end of vin-wise ----
    
df_temp3=pd.concat(xy_data_list, axis=0)
_=s3_write(bucket, 'res/2/batch_{}.csv'.format('POC_EGR_batch'), df_temp3)
#-------------------------

#from fsa import anova, pca, tsne

# Anova - F>1 : of df_temp3: box plots
# PCA of (# F>1) : scatterplots
# tsne - of concat(x_data_list) : scatterplots
# tsne - of (# F>1) : scatterplots
#--------------------------------

# import prediction_model as pm





1 MB1CTCFD6HAEG4844


/home/ec2-user/SageMaker/module1/statedwell.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3a['time']=pd.to_datetime(df3a['EVENT_UTC1'])
